# Financial ML System - Signal Generation

This notebook combines SVM and NN predictions to generate trading signals.

## 1. Setup

In [ ]:
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import tensorflow as tf
from tensorflow import keras

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

PROJECT_ROOT = Path('/content/financial-ml-system')
sys.path.insert(0, str(PROJECT_ROOT))

from src.utils.constants import DATA_DIR, MODELS_DIR, REGIME_NAMES, SIGNAL_BUY, SIGNAL_SELL, SIGNAL_HOLD
from src.utils.config_loader import config

print("Setup complete")

## 2. Load Models

In [ ]:
# Load SVM
svm_model = joblib.load(MODELS_DIR / 'svm' / 'svm_classifier.pkl')
svm_scaler = joblib.load(MODELS_DIR / 'svm' / 'scaler.pkl')

# Load NN
nn_model = keras.models.load_model(MODELS_DIR / 'nn' / 'nn_predictor.h5')
nn_scaler = joblib.load(MODELS_DIR / 'nn' / 'scaler.pkl')

print("Models loaded successfully")

## 3. Load Data

In [ ]:
ticker = config.get('data.default_ticker', 'SPY')
data = pd.read_csv(DATA_DIR / 'processed' / f"{ticker}_features.csv", 
                   index_col=0, parse_dates=True)

print(f"Loaded {len(data)} rows")
data.head()

## 4. Signal Generation Class

In [ ]:
class SignalGenerator:
    """Generate trading signals from model predictions."""
    
    def __init__(self, svm_model, svm_scaler, nn_model, nn_scaler, config):
        self.svm_model = svm_model
        self.svm_scaler = svm_scaler
        self.nn_model = nn_model
        self.nn_scaler = nn_scaler
        self.config = config
    
    def predict_regime(self, features_svm):
        """Predict market regime."""
        features_scaled = self.svm_scaler.transform(features_svm)
        regime = self.svm_model.predict(features_scaled)
        confidence = np.max(self.svm_model.decision_function(features_scaled), axis=1)
        return regime, confidence
    
    def predict_return(self, features_nn):
        """Predict returns."""
        features_scaled = self.nn_scaler.transform(features_nn)
        predicted_return = self.nn_model.predict(features_scaled, verbose=0).flatten()
        return predicted_return
    
    def generate_signal(self, regime, confidence, predicted_return):
        """Generate trading signal."""
        regime_threshold = self.config.get('trading.regime_threshold', 0.6)
        return_threshold_buy = self.config.get('trading.return_threshold_buy', 0.005)
        return_threshold_sell = self.config.get('trading.return_threshold_sell', -0.005)
        
        signals = []
        position_sizes = []
        
        for reg, conf, ret in zip(regime, confidence, predicted_return):
            # Normalize confidence to 0-1
            conf_normalized = min(abs(conf) / 2, 1.0)
            
            # Buy signal: Bull regime + positive return
            if (reg == 2 and ret > return_threshold_buy and conf_normalized > regime_threshold):
                signals.append(SIGNAL_BUY)
                position_sizes.append(conf_normalized)
            
            # Sell signal: Bear regime + negative return
            elif (reg == 0 and ret < return_threshold_sell and conf_normalized > regime_threshold):
                signals.append(SIGNAL_SELL)
                position_sizes.append(conf_normalized)
            
            # Hold: Everything else
            else:
                signals.append(SIGNAL_HOLD)
                position_sizes.append(0.0)
        
        return np.array(signals), np.array(position_sizes)

print("SignalGenerator class defined")

## 5. Generate Signals

In [ ]:
# Prepare features
svm_features = [
    'SMA_Ratio_5', 'SMA_Ratio_20', 'SMA_Ratio_50',
    'RSI', 'MACD', 'MACD_Hist',
    'BB_Width', 'BB_Position',
    'ATR_Pct', 'Volatility',
    'Volume_Ratio', 'ROC'
]

nn_features = [
    'Returns', 'SMA_Ratio_5', 'SMA_Ratio_20', 'SMA_Ratio_50',
    'RSI', 'MACD', 'MACD_Hist',
    'BB_Width', 'BB_Position',
    'ATR_Pct', 'Volatility',
    'Volume_Ratio', 'ROC', 'Momentum'
]

# Clean data
data_clean = data.dropna(subset=svm_features + nn_features)

X_svm = data_clean[svm_features].values
X_nn = data_clean[nn_features].values

# Initialize signal generator
signal_gen = SignalGenerator(svm_model, svm_scaler, nn_model, nn_scaler, config)

# Generate predictions
regime, confidence = signal_gen.predict_regime(X_svm)
predicted_return = signal_gen.predict_return(X_nn)

# Generate signals
signals, position_sizes = signal_gen.generate_signal(regime, confidence, predicted_return)

# Add to dataframe
data_clean = data_clean.copy()
data_clean['Regime'] = regime
data_clean['Confidence'] = confidence
data_clean['Predicted_Return'] = predicted_return
data_clean['Signal'] = signals
data_clean['Position_Size'] = position_sizes

print(f"Generated {len(signals)} signals")
print("\nSignal distribution:")
print(pd.Series(signals).value_counts())

## 6. Signal Analysis

In [ ]:
# Signal statistics
print("Signal Statistics")
print("=" * 50)

for signal in [SIGNAL_BUY, SIGNAL_SELL, SIGNAL_HOLD]:
    mask = data_clean['Signal'] == signal
    count = mask.sum()
    pct = count / len(data_clean) * 100
    avg_confidence = data_clean.loc[mask, 'Confidence'].mean()
    avg_position = data_clean.loc[mask, 'Position_Size'].mean()
    
    print(f"\n{signal}:")
    print(f"  Count: {count} ({pct:.1f}%)")
    print(f"  Avg Confidence: {avg_confidence:.4f}")
    print(f"  Avg Position Size: {avg_position:.4f}")

In [ ]:
# Visualize signals
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), height_ratios=[2, 1])

# Price and signals
ax1.plot(data_clean.index, data_clean['Close'], label='Price', linewidth=1, alpha=0.7)

# Plot buy signals
buy_mask = data_clean['Signal'] == SIGNAL_BUY
ax1.scatter(data_clean[buy_mask].index, data_clean[buy_mask]['Close'], 
           color='green', marker='^', s=100, label='Buy', alpha=0.7, zorder=5)

# Plot sell signals
sell_mask = data_clean['Signal'] == SIGNAL_SELL
ax1.scatter(data_clean[sell_mask].index, data_clean[sell_mask]['Close'],
           color='red', marker='v', s=100, label='Sell', alpha=0.7, zorder=5)

ax1.set_title('Trading Signals', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Position sizes
colors = ['green' if s == SIGNAL_BUY else 'red' if s == SIGNAL_SELL else 'gray' 
          for s in data_clean['Signal']]
ax2.bar(data_clean.index, data_clean['Position_Size'], color=colors, alpha=0.6)
ax2.set_title('Position Sizes', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('Position Size')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'trading_signals.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Regime and signal relationship
regime_signal = pd.crosstab(data_clean['Regime'], data_clean['Signal'])
regime_signal.index = [REGIME_NAMES[i] for i in regime_signal.index]

print("\nRegime vs Signal:")
print(regime_signal)

fig, ax = plt.subplots(figsize=(10, 6))
regime_signal.plot(kind='bar', stacked=True, ax=ax, color=['green', 'gray', 'red'])
ax.set_title('Signals by Market Regime', fontsize=14, fontweight='bold')
ax.set_xlabel('Market Regime')
ax.set_ylabel('Count')
ax.legend(title='Signal')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'signals_by_regime.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Save Signals

In [ ]:
# Save signals
signals_file = DATA_DIR / 'processed' / f"{ticker}_signals.csv"
data_clean.to_csv(signals_file)

print(f"Signals saved to: {signals_file}")
print(f"Shape: {data_clean.shape}")
print(f"Date range: {data_clean.index.min()} to {data_clean.index.max()}")

## 8. Create Signal Generation Module

In [ ]:
# Save signal generation module
signal_module = '''"""Signal generation module."""

import numpy as np

class SignalGenerator:
    """Generate trading signals from model predictions."""
    
    def __init__(self, svm_model, svm_scaler, nn_model, nn_scaler, config):
        self.svm_model = svm_model
        self.svm_scaler = svm_scaler
        self.nn_model = nn_model
        self.nn_scaler = nn_scaler
        self.config = config
    
    def predict_regime(self, features):
        features_scaled = self.svm_scaler.transform(features)
        regime = self.svm_model.predict(features_scaled)
        confidence = np.max(self.svm_model.decision_function(features_scaled), axis=1)
        return regime, confidence
    
    def predict_return(self, features):
        features_scaled = self.nn_scaler.transform(features)
        return self.nn_model.predict(features_scaled, verbose=0).flatten()
    
    def generate_signal(self, regime, confidence, predicted_return):
        regime_threshold = self.config.get('trading.regime_threshold', 0.6)
        return_buy = self.config.get('trading.return_threshold_buy', 0.005)
        return_sell = self.config.get('trading.return_threshold_sell', -0.005)
        
        signals = []
        position_sizes = []
        
        for reg, conf, ret in zip(regime, confidence, predicted_return):
            conf_norm = min(abs(conf) / 2, 1.0)
            
            if reg == 2 and ret > return_buy and conf_norm > regime_threshold:
                signals.append('BUY')
                position_sizes.append(conf_norm)
            elif reg == 0 and ret < return_sell and conf_norm > regime_threshold:
                signals.append('SELL')
                position_sizes.append(conf_norm)
            else:
                signals.append('HOLD')
                position_sizes.append(0.0)
        
        return np.array(signals), np.array(position_sizes)
'''

with open(PROJECT_ROOT / 'src' / 'signals' / 'generator.py', 'w') as f:
    f.write(signal_module)

print("Created: src/signals/generator.py")

## Signal Generation Complete

Next: Open `06_backtesting.ipynb`